# Analyse du sentiment (Imdb)

In [1]:
import pandas as pd

import numpy
# Inutile :
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import random

Using TensorFlow backend.


## Préparation et codage des données

In [2]:
data_train = pd.read_csv('imdb_train.csv')
data_test = pd.read_csv('imdb_test.csv')

In [3]:
print(data_train.Review[1])
print(data_train.Eval[1])

Okay, I know I shouldn't like this movie but I do. From Pat Morita's loveable interpretation of a Japanese stereotype to Jay Leno's annoying yell, I laughed throughout this movie.As long as you take into account that this is not the best movie in the world, it's a good mvie.<br /><br />My favorite part is Morita talking to his boss in Tokyo with the drinking a close second.
1


On sauvegarde les phrases avant transformation dans R_train, qu'on utilisera plus tard pour vérifier les prédictions

In [4]:
R_train = data_train.Review
Y_train = data_train.Eval

On utilise le *tokeniser* de Keras pour convertir les phrases en séquences de nombres (équivalent au codage avec *one_hot*)  
On retient les 5000 mots qui apparaissent le plus souvent (*top_words*)

In [5]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

top_words = 5000
tokenizer = Tokenizer(num_words=top_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(R_train)
X_train = tokenizer.texts_to_sequences(R_train)

In [6]:
print(R_train[1])
print(X_train[1])

Okay, I know I shouldn't like this movie but I do. From Pat Morita's loveable interpretation of a Japanese stereotype to Jay Leno's annoying yell, I laughed throughout this movie.As long as you take into account that this is not the best movie in the world, it's a good mvie.<br /><br />My favorite part is Morita talking to his boss in Tokyo with the drinking a close second.
[860, 10, 121, 10, 1609, 37, 11, 17, 18, 10, 78, 36, 3309, 2951, 4, 3, 857, 4348, 5, 3050, 613, 10, 1495, 466, 11, 17, 14, 193, 14, 22, 190, 80, 2640, 12, 11, 6, 21, 1, 115, 17, 8, 1, 179, 42, 3, 49, 7, 7, 58, 511, 170, 6, 659, 5, 24, 1407, 8, 16, 1, 2766, 3, 488, 330]


On transforme de même les données de test

In [7]:
R_test = data_test.Review
Y_test = data_test.Eval
X_test = tokenizer.texts_to_sequences(R_test)

On complète les séquences avec des 0 à gauche pour avoir des séquences de même longueur (ici 500)

In [8]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words, padding='post')
X_test = sequence.pad_sequences(X_test, maxlen=max_words, padding='post')

In [9]:
print(X_train[1])

[ 860   10  121   10 1609   37   11   17   18   10   78   36 3309 2951    4
    3  857 4348    5 3050  613   10 1495  466   11   17   14  193   14   22
  190   80 2640   12   11    6   21    1  115   17    8    1  179   42    3
   49    7    7   58  511  170    6  659    5   24 1407    8   16    1 2766
    3  488  330    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  

## Réseaux denses

On commence avec un modèle dense simple

In [10]:
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               4000250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 4,160,501
Trainable params: 4,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# Apprentissage
model.fit(X_train, Y_train, epochs=2, batch_size=128, verbose=2)
# Evaluation
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
20s - loss: 0.4911 - acc: 0.7439
Epoch 2/2
19s - loss: 0.1901 - acc: 0.9272
Accuracy: 86.42%


On calcule les prédictions

In [12]:
Y_mlp = model.predict(X_test)

### Test  
si prediction proche de 0 : avis négatif  
si prediction proche de 1 : avis positif

In [19]:
ir = random.randint(1, X_test.shape[0])
print(R_test[ir])
print(Y_mlp[ir])

This movie was sheer, slow, plodding torture. Not being a fan of slasher films, and preferring classic horror, I may not be the best judge. Slasher fans may enjoy this as an early entry into the slasher genre. I really would like the 90 minutes I spent watching this back. The only reasons to watch this are 1) a young Vanna White (pre Wheel of Fortune) who has about 3 minutes of total screen time. 2) an early role by scream queen Linnea Quigley (with nudity), and legendary beauty EJ Peaker as the school secretary, who added a bit of humor and sex appeal. Otherwise, an 80's band called Felony makes an appearance, which while interesting, drags on WAY too long. Just like the movie.....I had to keep checking how much time was left and couldn't wait to "Graduate" to a better film.
[ 0.0972731]


## Réseaux convolutionnels

In [20]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

On utilise une couche convolutionnelle à une dimension

In [21]:
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=10, input_shape=(32, 1), activation="relu"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 491, 32)           10272     
_________________________________________________________________
flatten_2 (Flatten)          (None, 15712)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15713     
Total params: 185,985
Trainable params: 185,985
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# Apprentissage
model.fit(X_train, Y_train, epochs=10, batch_size=128, verbose=2)
# Evaluation
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
22s - loss: 0.0072 - acc: 0.9994
Epoch 2/10
21s - loss: 0.0035 - acc: 1.0000
Epoch 3/10
21s - loss: 0.0022 - acc: 1.0000
Epoch 4/10
21s - loss: 0.0015 - acc: 1.0000
Epoch 5/10
21s - loss: 0.0011 - acc: 1.0000
Epoch 6/10
21s - loss: 8.0491e-04 - acc: 1.0000
Epoch 7/10
21s - loss: 6.2550e-04 - acc: 1.0000
Epoch 8/10
21s - loss: 5.0742e-04 - acc: 1.0000
Epoch 9/10
21s - loss: 4.1021e-04 - acc: 1.0000
Epoch 10/10
21s - loss: 3.3958e-04 - acc: 1.0000
Accuracy: 86.24%


In [24]:
Y_cnn1 = model.predict(X_test)

In [25]:
ir = random.randint(1, X_test.shape[0])
print(R_test[ir])
print(Y_cnn1[ir])

The first time I saw a commercial for this show was when my sisters were watching the Kim Possible movie. The commercial showed Sadie and her friend discussing the meaning of the word nothing.It is one of the stupidest commercials I've ever seen. Basically, they go back and forth with lines like "Nothing is a thing, so technically nothing is something,". When I saw that, I figured it would be yet another lame Lizzie Maguire knockoff by Disney. But I had no idea how bad.<br /><br />Fast forward about 3 weeks, when my sister turns on the T.V. Naturally Sadie happened to be on. What I saw had to be one of the most unintentionally funny shows I've ever seen. How'd it go? Something like this:<br /><br />Sadie, a vegetarian tree hugger, has an incredibly unhealthy,obsessive crush on the very monotone and poorly acted acted out Owen. For some reason, her friend Margret decides that Owen needs to be "tested" to see if he is as good as he seems. What exactly do these tests involve? Well, one th

Avec un réseau un peu plus complexe :

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 128, input_length=max_words))
model.add(Conv1D(filters=128, kernel_size=10, input_shape=(128, 1), activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
# Apprentissage
model.fit(X_train, Y_train, epochs=10, batch_size=128, verbose=2)
# Evaluation
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
Y_cnn2 = model.predict(X_test)

In [ ]:
ir = random.randint(1, X_test.shape[0])
print(R_test[ir])
print(Y_cnn2[ir])

## Réseaux récurrents (LSTM)

In [ ]:
from keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Attention :l'apprentissage avec des LSTM peut être assez long ...

In [ ]:
# Apprentissage
model.fit(X_train, Y_train, epochs=10, batch_size=128, verbose=2)
# Evaluation
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Pour les installation avec *Tensorflow GPU*, on peut utiliser *CuDNNLSTM*, beaucoup plus rapide :

In [ ]:
from keras.layers import CuDNNLSTM, BatchNormalization

In [ ]:
model = Sequential()
model.add(Embedding(top_words, 100, input_length=max_words))
model.add(CuDNNLSTM(128))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
# Apprentissage
model.fit(X_train, Y_train, epochs=10, batch_size=128, verbose=2)
# Evaluation
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

## Exercice

Tester les différentes approches sur le dataset des Newsgroups Reuters  
https://keras.io/datasets/#reuters-newswire-topics-classification  
  
  
Il s'agit de classer les news dans un des 46 groupes

In [26]:
from keras.datasets import reuters  
top_words = 30000  
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=top_words)

Exception: URL fetch failure on https://s3.amazonaws.com/text-datasets/reuters.npz: None -- [Errno 11001] getaddrinfo failed

La lecture directe du dataset ne nécessite pas la phase de tokenisation/codage

**Indication** : on doit ici réaliser une classification dans des catégories multiples (un peu différent de la décision 0/1 de l'exemple Imdb). On pourra s'inspirer de l'exemple *mnist*

In [ ]:
print(R_train[1])
print(X_train[1])